In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
#import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import cv2
from create_model_trang import create_cnn_model_trang
from keras.callbacks import ModelCheckpoint  

from keras.applications import inception_v3
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inception_v3_preprocessor

from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

from sklearn.model_selection import train_test_split

from tqdm import tqdm

from os import makedirs
from os.path import expanduser, exists, join

Using TensorFlow backend.


In [3]:
# Use keras pretrained models
# Source: https://www.kaggle.com/gaborfodor/resnet50-example

!ls ../keras-pretrained-models/

cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)
    
!cp ../keras-pretrained-models/*notop* ~/.keras/models/
!cp ../keras-pretrained-models/imagenet_class_index.json ~/.keras/models/
!cp ../keras-pretrained-models/resnet50* ~/.keras/models/

imagenet_class_index.json
inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5
inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5
inception_v3_weights_tf_dim_ordering_tf_kernels.h5
inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Kuszma.JPG
resnet50_weights_tf_dim_ordering_tf_kernels.h5
resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
xception_weights_tf_dim_ordering_tf_kernels.h5
xception_weights_tf_dim_ordering_tf_kernels_notop.h5


In [4]:
# Set train and test folders
train_folder = '../train/'
test_folder = '../test/'

In [5]:
training_data = pd.read_csv('../labels.csv')
training_data['image_path'] = training_data.apply( lambda x: (train_folder + x["id"] + ".jpg" ), axis=1)
training_data.head(10)

,id,breed,image_path
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,../train/000bec180eb18c7604dcecc8fe0dba07.jpg
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,../train/001513dfcb2ffafc82cccf4d8bbaba97.jpg
2,001cdf01b096e06d78e9e5112d419397,pekinese,../train/001cdf01b096e06d78e9e5112d419397.jpg
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,../train/00214f311d5d2247d5dfe4fe24b2303d.jpg
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,../train/0021f9ceb3235effd7fcde7f7538ed62.jpg
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier,../train/002211c81b498ef88e1b40b9abf84e1d.jpg
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier,../train/00290d3e1fdd27226ba27a8ce248ce85.jpg
7,002a283a315af96eaea0e28e7163b21b,borzoi,../train/002a283a315af96eaea0e28e7163b21b.jpg
8,003df8b8a8b05244b1d920bb6cf451f9,basenji,../train/003df8b8a8b05244b1d920bb6cf451f9.jpg
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound,../train/0042188c895a2f14ef64a918ed9c7b64.jpg


In [7]:
# Group image paths into an array to train on
train_data = np.array([img_to_array(load_img(img, target_size=(224, 224))) for img in training_data['image_path'].values.tolist()]).astype('float32')

In [8]:
# Split the data into train and validation. Since we only have train and validation folders, need to divide train into training and validation sets. 
# Save validation folder for later testing
x_train, x_validation, y_train, y_validation = train_test_split(train_data, 
                                                                training_data["breed"], 
                                                                test_size=0.2, 
                                                                stratify=np.array(training_data["breed"]), 
                                                                random_state=1234)

In [9]:
# Need to convert the train and validation labels into one hot encoded format
y_train = pd.get_dummies(y_train.reset_index(drop=True)).as_matrix()
y_validation = pd.get_dummies(y_validation.reset_index(drop=True)).as_matrix()


In [10]:
# Create train generator.
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, 
                                   horizontal_flip = True)
train_generator = train_datagen.flow(x_train, y_train, shuffle=False, batch_size=20, seed=100)

In [11]:
# Create validation generator
val_datagen = ImageDataGenerator(rescale = 1./255)
val_generator = train_datagen.flow(x_validation, y_validation, shuffle=False, batch_size=20, seed=100)

In [17]:
# CNN from scratch
model = create_cnn_model_trang()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='out_trang/weights.newbestaugmented.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)
batch_size = 20
model.fit_generator(
        train_generator,
        steps_per_epoch=2000// batch_size,
        epochs=10,
        validation_data=val_generator,
        validation_steps=800 // batch_size,
        callbacks=[checkpointer])

Epoch 1/10
100/100 [==============================] - ETA: 11:00 - loss: 4.8476 - acc: 0.0000e+ - ETA: 8:09 - loss: 4.7791 - acc: 0.0000e+00 - ETA: 7:03 - loss: 4.8184 - acc: 0.0000e+0 - ETA: 6:26 - loss: 4.8291 - acc: 0.0000e+0 - ETA: 5:59 - loss: 4.8555 - acc: 0.0000e+0 - ETA: 5:43 - loss: 4.8118 - acc: 0.0083    - ETA: 5:28 - loss: 4.8275 - acc: 0.007 - ETA: 5:16 - loss: 4.8918 - acc: 0.006 - ETA: 5:07 - loss: 4.8941 - acc: 0.011 - ETA: 4:59 - loss: 4.8680 - acc: 0.010 - ETA: 4:51 - loss: 4.8776 - acc: 0.009 - ETA: 4:44 - loss: 4.8885 - acc: 0.008 - ETA: 4:37 - loss: 4.8808 - acc: 0.007 - ETA: 4:32 - loss: 4.8673 - acc: 0.007 - ETA: 4:26 - loss: 4.8766 - acc: 0.006 - ETA: 4:21 - loss: 4.8561 - acc: 0.006 - ETA: 4:17 - loss: 4.8807 - acc: 0.005 - ETA: 4:12 - loss: 4.8979 - acc: 0.005 - ETA: 4:08 - loss: 4.8965 - acc: 0.007 - ETA: 4:03 - loss: 4.8892 - acc: 0.010 - ETA: 3:59 - loss: 4.9025 - acc: 0.009 - ETA: 3:56 - loss: 4.9129 - acc: 0.009 - ETA: 3:53 - loss: 4.9297 - acc: 0.008 - E


Epoch 00003: val_loss improved from 4.82928 to 4.76593, saving model to out_trang/weights.newbestaugmented.from_scratch.hdf5
Epoch 4/10
100/100 [==============================] - ETA: 5:16 - loss: 4.6303 - acc: 0.0000e+0 - ETA: 5:13 - loss: 4.6580 - acc: 0.0000e+0 - ETA: 5:04 - loss: 4.6795 - acc: 0.0000e+0 - ETA: 4:55 - loss: 4.6999 - acc: 0.0125    - ETA: 4:48 - loss: 4.7044 - acc: 0.010 - ETA: 4:44 - loss: 4.7061 - acc: 0.016 - ETA: 4:39 - loss: 4.7091 - acc: 0.014 - ETA: 4:34 - loss: 4.7144 - acc: 0.012 - ETA: 4:29 - loss: 4.7044 - acc: 0.016 - ETA: 4:26 - loss: 4.7162 - acc: 0.015 - ETA: 4:21 - loss: 4.7202 - acc: 0.013 - ETA: 4:18 - loss: 4.7117 - acc: 0.016 - ETA: 4:16 - loss: 4.7104 - acc: 0.015 - ETA: 4:12 - loss: 4.7185 - acc: 0.014 - ETA: 4:08 - loss: 4.6989 - acc: 0.013 - ETA: 4:05 - loss: 4.6803 - acc: 0.012 - ETA: 4:02 - loss: 4.6725 - acc: 0.011 - ETA: 3:58 - loss: 4.6704 - acc: 0.011 - ETA: 3:55 - loss: 4.6795 - acc: 0.010 - ETA: 3:52 - loss: 4.6783 - acc: 0.010 - ETA:

Epoch 00006: val_loss improved from 4.76593 to 4.51640, saving model to out_trang/weights.newbestaugmented.from_scratch.hdf5
Epoch 7/10
100/100 [==============================] - ETA: 5:18 - loss: 4.8169 - acc: 0.0000e+0 - ETA: 4:58 - loss: 4.5896 - acc: 0.0000e+0 - ETA: 4:47 - loss: 4.6226 - acc: 0.0000e+0 - ETA: 4:41 - loss: 4.5976 - acc: 0.0000e+0 - ETA: 4:35 - loss: 4.5663 - acc: 0.0000e+0 - ETA: 4:31 - loss: 4.5453 - acc: 0.0083    - ETA: 4:27 - loss: 4.5289 - acc: 0.007 - ETA: 4:23 - loss: 4.5460 - acc: 0.006 - ETA: 4:20 - loss: 4.5491 - acc: 0.011 - ETA: 4:16 - loss: 4.5295 - acc: 0.010 - ETA: 4:13 - loss: 4.4968 - acc: 0.027 - ETA: 4:10 - loss: 4.4894 - acc: 0.025 - ETA: 4:07 - loss: 4.4696 - acc: 0.026 - ETA: 4:04 - loss: 4.4579 - acc: 0.028 - ETA: 4:01 - loss: 4.4358 - acc: 0.030 - ETA: 3:58 - loss: 4.4248 - acc: 0.031 - ETA: 3:55 - loss: 4.4172 - acc: 0.032 - ETA: 3:52 - loss: 4.4235 - acc: 0.030 - ETA: 3:49 - loss: 4.4301 - acc: 0.031 - ETA: 3:46 - loss: 4.4315 - acc: 0.030


Epoch 00009: val_loss did not improve from 4.32351
Epoch 10/10
100/100 [==============================] - ETA: 6:00 - loss: 4.4570 - acc: 0.0000e+0 - ETA: 5:15 - loss: 4.2911 - acc: 0.0750    - ETA: 5:00 - loss: 4.2683 - acc: 0.066 - ETA: 4:52 - loss: 4.3261 - acc: 0.062 - ETA: 4:45 - loss: 4.2942 - acc: 0.050 - ETA: 4:39 - loss: 4.3029 - acc: 0.050 - ETA: 4:35 - loss: 4.3465 - acc: 0.042 - ETA: 4:32 - loss: 4.3219 - acc: 0.050 - ETA: 4:29 - loss: 4.3148 - acc: 0.050 - ETA: 4:28 - loss: 4.3077 - acc: 0.050 - ETA: 4:26 - loss: 4.3207 - acc: 0.045 - ETA: 4:22 - loss: 4.3261 - acc: 0.045 - ETA: 4:21 - loss: 4.2954 - acc: 0.046 - ETA: 4:19 - loss: 4.2884 - acc: 0.042 - ETA: 4:19 - loss: 4.2932 - acc: 0.043 - ETA: 4:16 - loss: 4.3005 - acc: 0.040 - ETA: 4:13 - loss: 4.3019 - acc: 0.038 - ETA: 4:09 - loss: 4.3169 - acc: 0.038 - ETA: 4:05 - loss: 4.3307 - acc: 0.036 - ETA: 4:01 - loss: 4.3533 - acc: 0.037 - ETA: 3:58 - loss: 4.3450 - acc: 0.040 - ETA: 3:55 - loss: 4.3411 - acc: 0.038 - ETA: 

In [ ]:
# Get the InceptionV3 model as the base model for training
base_model = InceptionV3(weights = 'imagenet', include_top = False, input_shape=(224, 224, 3))
# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# Add a fully-connected layer and a logistic layer 
x = Dense(512, activation='relu')(x)
predictions = Dense(120, activation='softmax')(x)

model = Model(inputs = base_model.input, outputs = predictions)
# first: train only the top layers i.e. freeze all convolutional InceptionV3 layers
# for layer in base_model.layers:
#    layer.trainable = False
    
checkpointer = ModelCheckpoint(filepath='out_trang/weights.pretrained.hdf5', 
                               save_best_only=True)
# Compile with Adam
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
model.fit_generator(train_generator,
                      steps_per_epoch = 818,
                      validation_data = val_generator,
                      validation_steps = 205,
                      epochs = 20,
                      verbose = 2, callbacks=[checkpointer])

Epoch 1/20


In [17]:
# Use the sample submission file to set up the test data - x_test
test_data = pd.read_csv('../sample_submission.csv')
# Create the x_test
x_test = []
for i in tqdm(test_data['id'].values):
    img = cv2.imread('../test/{}.jpg'.format(i))
    x_test.append(cv2.resize(img, (224, 224)))
# Turn into an array
x_test = np.array(x_test, np.float32) / 255.

100%|██████████| 10357/10357 [02:04<00:00, 80.91it/s]


MemoryError: 

In [ ]:
# Predict x_test
predictions = model.predict(x_test, verbose=2)
# Set column names to those generated by the one-hot encoding 
one_hot_ys = pd.get_dummies(training_data["breed"], sparse = True)
col_names = one_hot_ys.columns.values